In [ ]:
import os 
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, BitsAndBytesConfig
from lm_eval.api.instance import Instance
import tqdm.notebook as tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
categories_dict = {
     'Smart Devices':                                  1,
     'Wearable devices':                               2,
     'Time & location':                                3,
     'Online services':                                4,
     'Personal data managers and schedulers':          5,
     'Iot Hubs/Integration solutions':                 6,
     'Automobiles':                                    7,
     'Social networking, blogging, sharing platforms': 8,
     'Cloud storage':                                  9,
     'Messaging, team collaboration, VoIP':            10,
     'Smartphone applications':                        11,
     'RSS and recommendation systems':                 12,
     'Mail services':                                  13,
     'Other':                                          14,
}

In [ ]:
class TextCategoryModel(nn.Module):
    def __init__(self, num_categories, hidden_dim=128, text_embedding_dim=384):
        super(TextCategoryModel, self).__init__()
        self.text_embedding_dim = text_embedding_dim  # From all-MiniLM-L6-v2
        self.fc_text = nn.Linear(self.text_embedding_dim, hidden_dim)  # From 384 to hidden_dim
        self.fc_combined = nn.Linear(hidden_dim, hidden_dim)  # Keep the same dimension for next layer
        self.output_layer = nn.Linear(hidden_dim, num_categories)  # Output layer for multi-class

    def forward(self, text_embedding):
        # Pass through text embedding layer
        text_out = F.relu(self.fc_text(text_embedding))
        combined_out = F.relu(self.fc_combined(text_out))  # Apply ReLU activation after combining
        output = self.output_layer(combined_out)  # Logits for multi-class
        return output

def get_text_embedding(text, sentence_model):
    # Get text embeddings from the sentence transformer
    #try:
    embedding = sentence_model.encode(text, convert_to_tensor=True, device=device, show_progress_bar=False)
    #except:
    #embedding = sentence_model.encode(text, convert_to_tensor=True, device=device)
    return embedding

def classify_text(text, category_classifier, sentence_model):
    # Get text embedding
    text_embedding = get_text_embedding(text, sentence_model)

    with torch.no_grad():
        outputs = category_classifier(text_embedding.unsqueeze(0).to(device))  # Add batch dimension
        predicted_class = torch.argmax(outputs, dim=1).item()  # Get the index of the predicted class

    return predicted_class + 1

def predict_joint_probability(sub_dialogue, model, tokenizer):
    input_ids = tokenizer(sub_dialogue, return_tensors="pt").input_ids.to(device)

    # Create an Instance object
    instance = Instance(request_type="loglikelihood_rolling", doc={}, arguments=(sub_dialogue,), idx=0)

    # Compute the loglikelihood
    log_likelihood = model.loglikelihood_rolling([instance], True)[0]

    return log_likelihood, input_ids.size(1)

def predict_probability_perplexity(log_likelihood, num_token):
    return math.exp(-log_likelihood / num_token)

def compute_perplexity(text, model, tokenizer):
    log_likelihood, num_token = predict_joint_probability(text, model, tokenizer)
    perplexity = predict_probability_perplexity(log_likelihood, num_token)

    return perplexity, log_likelihood, num_token

def topk_service(text, list, sentence_model, k):
    scored = []

    for i in range(len(list)):
        text_embedding = get_text_embedding(text, sentence_model)
        service_embedding = get_text_embedding(list[i], sentence_model)
        score = sentence_model.similarity(text_embedding, service_embedding)
        scored.append((score.item(), list[i]))
    scored.sort(reverse=True)

    return scored[:k]

def get_function(services, functions):
    results = {}
    for i in range(len(services)):
        service = services[i][1]
        if service in functions:
            results[service] = functions[service]

    return results

def get_combinations(triggers, actions):
    for trigger_service, trigger_functions in triggers.items():
        for trigger_function in trigger_functions:
            for action_service, action_functions in actions.items():
                for action_function in action_functions:
                    yield str('IF ' + trigger_service) + ' ' + str(trigger_function) + ' THEN ' + str(action_service) + ' ' + str(action_function)

def load_model(adapter_model_id='adapter/recipe_adapter'):
    tokenizer = AutoTokenizer.from_pretrained('LLM NAME')
    quantization_config = BitsAndBytesConfig(load_in_8bit=True)
    model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path='LLM NAME',
        quantization_config=quantization_config,
        device_map="auto"
    )

    tapir_model = PeftModel.from_pretrained(model, model_id=adapter_model_id, is_trainable=True)

    print(tapir_model.print_trainable_parameters())

    return tapir_model, tokenizer

model, tokenizer = load_model('adapter_name')

def get_topk_recipe(generated, combinations, trigger_embedder, action_embedder, k):
    scored = []
    try:
        trigger_generated = generated.split('THEN', 1)[0].strip()
        action_generated = 'THEN' + generated.split('THEN', 1)[1].strip()
    except:
        return scored

    trigger_generated_embedding = get_text_embedding(trigger_generated, trigger_embedder)
    action_generated_embedding = get_text_embedding(action_generated, action_embedder)

    for combination in combinations:
        trigger_combination = combination.split('THEN', 1)[0].strip()
        action_combination = 'THEN' + combination.split('THEN', 1)[1].strip()
        trigger_embedding = get_text_embedding(trigger_combination, trigger_embedder)
        action_embedding = get_text_embedding(action_combination, action_embedder)

        score_trigger = trigger_embedder.similarity(trigger_generated_embedding, trigger_embedding)
        score_action = action_embedder.similarity(action_generated_embedding, action_embedding)
        scored.append(compute_perplexity(combination, model, tokenizer) / score_trigger.item() * score_action.item(), combination)
        #scored.append((score_trigger.item() * score_action.item(), combination))
    scored.sort(reverse=True)
    return scored[:k]

def compute_mrr(list, match, k):
    for i in range(min(k, len(list))): 
        if list[i][1] == match:
            return 1 / (i + 1)
    return 0

In [ ]:
import json
one_shot = True
# Load generated test set
gold_generated_crg_set = pd.read_csv('../../data/generated/generated_recipe_gold.csv')
noisy_generated_crg_set = pd.read_csv('../../data/generated/generated_recipe_noisy.csv')

gold_generated_icg_trigger_set = pd.read_csv('../../data/generated/generated_trigger_gold.csv')
noisy_generated_icg_trigger_set = pd.read_csv('../../data/generated/generated_trigger_noisy.csv')

gold_generated_icg_action_set = pd.read_csv('../../data/generated/generated_action_gold.csv')
noisy_generated_icg_action_set = pd.read_csv('../../data/generated/generated_action_noisy.csv')

# Load one shot
if one_shot:
    oneshot = pd.read_csv('../../data/dataset/gold/test_recipe_one_shot.csv')
    for j, row_j in gold_generated_crg_set.iterrows():
        found = False
        for i, row_i in oneshot.iterrows():
            if row_j['input'] == row_i['input']:
                found = True
                break
        if not found:
            gold_generated_crg_set.drop(j, inplace=True)
            gold_generated_icg_trigger_set.drop(j, inplace=True)
            gold_generated_icg_action_set.drop(j, inplace=True)

gold_generated_icg_trigger_set.reset_index(drop=True, inplace=True)
gold_generated_icg_action_set.reset_index(drop=True, inplace=True)
gold_generated_crg_set.reset_index(drop=True, inplace=True)
print(len(gold_generated_crg_set))
print(len(gold_generated_icg_trigger_set))
print(len(gold_generated_icg_action_set))
# Service list
action_services = pd.read_csv(f"../../data/services/action_services.csv")
trigger_services = pd.read_csv(f"../../data/services/trigger_services.csv")

# Functionality list
with open("../../data/services/action_functions.json") as json_file:
    action_functions = json.load(json_file)
    
with open("../../data/services/trigger_functions.json") as json_file:
    trigger_functions = json.load(json_file)

In [ ]:
gold_generated_crg_set.to_csv('../../data/generated/generated_recipe_gold_one_shot.csv', index=False)
gold_generated_icg_trigger_set.to_csv('../../data/generated/generated_trigger_gold_one_shot.csv', index=False)
gold_generated_icg_action_set.to_csv('../../data/generated/generated_action_gold_one_shot.csv', index=False)

In [ ]:
def topk_service(text, list, sentence_model, k):
    scored = []

    service_embeddings = get_text_embedding(list, sentence_model)
    text_embedding = get_text_embedding(text, sentence_model)

    for i in range(len(list)):
        score = sentence_model.similarity(text_embedding, service_embeddings[i])
        scored.append((score.item(), list[i]))
    scored.sort(reverse=True)

    return scored[:k]

def get_topk_recipe(generated, combinations, trigger_embedder, action_embedder, k):
    scored = []
    try:
        trigger_generated = generated.split('THEN', 1)[0].strip()
        action_generated = 'THEN' + generated.split('THEN', 1)[1].strip()
    except:
        return scored

    trigger_generated_embedding = get_text_embedding(trigger_generated, trigger_embedder)
    action_generated_embedding = get_text_embedding(action_generated, action_embedder)

    trigger_combinations = []
    action_combinations = []
    temp_combination = []
    for combination in combinations:
        trigger_combinations.append(combination.split('THEN', 1)[0].strip())
        action_combinations.append('THEN' + combination.split('THEN', 1)[1].strip())
        temp_combination.append(combination)

    trigger_embeddings = get_text_embedding(trigger_combinations, trigger_embedder)
    action_embeddings = get_text_embedding(action_combinations, action_embedder)

    i = 0
    for combination in temp_combination:
        score_trigger = trigger_embedder.similarity(trigger_generated_embedding, trigger_embeddings[i]).item()
        score_action = action_embedder.similarity(action_generated_embedding, action_embeddings[i]).item()
        scored.append((score_trigger * score_action, combination))
        i+=1
    scored.sort(reverse=True)
    return scored[:k]

In [ ]:
from sentence_transformers import SentenceTransformer

name = 'NAME'
target = 'default'

if target == 'default':
    action_embedder = SentenceTransformer(f'{name}').to(device)
    trigger_embedder = SentenceTransformer(f'{name}').to(device)
else:
    action_embedder = SentenceTransformer(f'../../models/{name}/action_{target}_{name}_embedder').to(device)
    trigger_embedder = SentenceTransformer(f'../../models/{name}/trigger_{target}_{name}_embedder').to(device)

num_categories = 14
trigger_classifier = TextCategoryModel(num_categories,text_embedding_dim=trigger_embedder.get_sentence_embedding_dimension()).to(device)
action_classifier = TextCategoryModel(num_categories,text_embedding_dim=action_embedder.get_sentence_embedding_dimension()).to(device)

trigger_classifier.load_state_dict(torch.load(f'../../models/{name}/trigger_{target}_classifier/trigger_classifier.pth'))
action_classifier.load_state_dict(torch.load(f'../../models/{name}/action_{target}_classifier/action_classifier.pth'))

In [ ]:
output_df = pd.DataFrame(columns=['top', 'output'])

In [ ]:
exact_match = 0
mrr_3 = 0.0
mrr_5 = 0.0
verbose = False
pbar =  tqdm.tqdm(gold_generated_crg_set.iterrows(), total=len(gold_generated_crg_set), desc="Processing Rows")

for i, row in pbar:
    if row['generated'] == row['output']:
        temp_df = pd.DataFrame({'top': [row['generated']], 'output': [row['output']]})
        output_df = pd.concat([output_df, temp_df], ignore_index=True)
        exact_match += 1
        mrr_3 += 1
        mrr_5 += 1
        continue

    try:
        trigger, action = row['generated'].split('THEN', 1)
        trigger = trigger.replace('IF','').strip()
        action = action.strip()
    except:
        trigger = row['generated']
        action = row['generated']

    trigger_class = classify_text(trigger, trigger_classifier, trigger_embedder)
    action_class = classify_text(action, action_classifier, action_embedder)

    trigger_list = trigger_services[trigger_services['category'] == trigger_class]
    action_list = action_services[action_services['category'] == action_class]
    topk_trigger = topk_service(trigger, trigger_list['service'].tolist(), trigger_embedder, 3)
    topk_action = topk_service(action, action_list['service'].tolist(), action_embedder, 3)
    topk_trigger = get_function(topk_trigger, trigger_functions)
    topk_action = get_function(topk_action, action_functions)
    topk_recipe = get_topk_recipe(row['generated'], get_combinations(topk_trigger, topk_action), trigger_embedder, action_embedder, 5)

    if not topk_recipe:
        exact_match += 1
        mrr_3 += 1
        mrr_5 += 1
        continue

    if topk_recipe[0][1] == row['output']:
        exact_match += 1

    temp_df = pd.DataFrame({'top': topk_recipe[0][1], 'output': [row['output']]})
    output_df = pd.concat([output_df, temp_df], ignore_index=True)

    mrr_3 += compute_mrr(topk_recipe, row['output'], 3)
    mrr_5 += compute_mrr(topk_recipe, row['output'], 5)
    output_df.to_csv('../../error_analysis/one_shot.csv', index=False)

    pbar.set_postfix(exact_match=exact_match/(i+1), mrr_3=mrr_3/(i+1), mrr_5=mrr_5/(i+1))

output_df.to_csv('../../error_analysis/one_shot_final.csv', index=False)

In [ ]:
print('Exact Match:', exact_match / len(gold_generated_crg_set))
print('MRR@3:', mrr_3 / len(gold_generated_crg_set))
print('MRR@5:', mrr_5 / len(gold_generated_crg_set))

In [ ]:
exact_match = 0
mrr_3 = 0.0
mrr_5 = 0.0
verbose = False
pbar =  tqdm.tqdm(zip(gold_generated_icg_trigger_set.iterrows(), gold_generated_icg_action_set.iterrows()), total=len(gold_generated_icg_trigger_set), desc="Processing Rows")
counter_split = 0
counter_eq = 0
for (i, t_row), (j, a_row) in pbar:
    try:
        trigger = t_row['generated'].replace('TRIGGER SERVICE: ', '').strip().replace(', TRIGGER EVENT: ', ' ')
        action = a_row['generated'].replace('ACTION SERVICE: ', '').strip().replace(', ACTION EVENT: ', ' ')
    except:
        exact_match += 1
        mrr_3 += 1
        mrr_5 += 1
        counter_split += 1
        continue

    generated = 'IF ' + str(trigger) + ' THEN ' + str(action)

    if generated == gold_generated_crg_set.iloc[i]['output']:
        exact_match += 1
        mrr_3 += 1
        mrr_5 += 1
        counter_eq += 1
        continue


    trigger_class = classify_text(trigger, trigger_classifier, trigger_embedder)
    action_class = classify_text(action, action_classifier, action_embedder)
    trigger_list = trigger_services[trigger_services['category'] == trigger_class]
    action_list = action_services[action_services['category'] == action_class]
    topk_trigger = topk_service(trigger, trigger_list['service'].tolist(), trigger_embedder, 3)
    topk_action = topk_service(action, action_list['service'].tolist(), action_embedder, 3)
    topk_trigger = get_function(topk_trigger, trigger_functions)
    topk_action = get_function(topk_action, action_functions)
    topk_recipe = get_topk_recipe(generated, get_combinations(topk_trigger, topk_action), trigger_embedder, action_embedder, 5)

    if not topk_recipe:
        exact_match += 1
        mrr_3 += 1
        mrr_5 += 1
        continue

    if topk_recipe[0][1] == gold_generated_crg_set.iloc[i]['output']:
        exact_match += 1

    mrr_3 += compute_mrr(topk_recipe, gold_generated_crg_set.iloc[i]['output'], 3)
    mrr_5 += compute_mrr(topk_recipe, gold_generated_crg_set.iloc[i]['output'], 5)

    pbar.set_postfix(exact_match=exact_match/(i+1), mrr_3=mrr_3/(i+1), mrr_5=mrr_5/(i+1), counter_eq=counter_eq)

In [ ]:
print('Exact Match:', exact_match / len(gold_generated_crg_set))
print('MRR@3:', mrr_3 / len(gold_generated_crg_set))
print('MRR@5:', mrr_5 / len(gold_generated_crg_set))